In [68]:
import tensorflow.keras.layers as tkl

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD

In [69]:
class All_Conv2D(tkl.Layer):
    def __init__(self, filters: int, kernel_size: tuple[int, int],
                 strides: int, padding: str = "same"):
        """
        将卷积和批量标准化和激活函数整合的全卷积类

        :param filters: 核个数
        :param kernel_size: 核大小
        :param strides: 步长
        :param padding: 填充
        """
        super(All_Conv2D, self).__init__()
        self.conv2d = tkl.Conv2D(filters=filters,
                                 kernel_size=kernel_size,
                                 strides=strides,
                                 padding=padding)
        self.bn = tkl.BatchNormalization()
        self.active_fun = tkl.ReLU()

    def call(self, inputs, *args, **kwargs):
        output = self.conv2d(inputs)
        output = self.bn(output)
        output = self.active_fun(output)
        return output


class Preprocess(tkl.Layer):
    def __init__(self):
        super(Preprocess, self).__init__()
        self.a_con1 = All_Conv2D(filters=64, kernel_size=(3, 3),
                                strides=1)
        self.max_pool1 = tkl.MaxPooling2D(pool_size=(3, 3),
                                         strides=2, padding="same")

        self.a_con2 = All_Conv2D(filters=192, kernel_size=(3, 3),
                                 strides=1)
        self.max_pool2 = tkl.MaxPooling2D(pool_size=(3, 3),
                                          strides=2, padding="same")

    def call(self, inputs, *args, **kwargs):
        output = self.a_con1(inputs)
        output = self.max_pool1(output)
        output = self.a_con2(output)
        output = self.max_pool2(output)
        return output


class Inception_model1(tkl.Layer):
    def __init__(self):
        super(Inception_model1, self).__init__()
        # branch 0
        self.a_con_b0 = All_Conv2D(filters=64, kernel_size=(1, 1),
                                strides=1)

        # branch 1
        self.a_con_b1 = All_Conv2D(filters=64, kernel_size=(3, 3),
                                   strides=1)

        # branch 2
        self.a_pool_b2 = tkl.AvgPool2D(pool_size=(3, 3), strides=1,
                                          padding="same")
        self.a_con_b2 = All_Conv2D(filters=32, kernel_size=(1, 1),
                                   strides=1)

    def call(self, inputs, *args, **kwargs):
        b0 = self.a_con_b0(inputs)

        b1 = self.a_con_b1(inputs)

        b2 = self.a_pool_b2(inputs)
        b2 = self.a_con_b2(b2)
        # print(b0.shape, b1.shape, b2.shape)
        output = tkl.concatenate([b0, b1, b2], axis=-1)
        return output


class Inception_V1(Model):
    def get_config(self):
        super(Inception_V1, self).get_config()

    def __init__(self, num_classes: int):
        """
        v1模型

        :param num_classes: 类别数
        """
        super(Inception_V1, self).__init__()
        self.preprocess = Preprocess()

        self.block = Sequential([Inception_model1()])
        self.avg_pool = tkl.AvgPool2D(pool_size=(8, 8),
                                      strides=1,
                                      padding="valid")
        self.dropout = tkl.Dropout(0.5)
        self.flatten = tkl.Flatten()
        self.fc1 = tkl.Dense(256)
        self.ac1 = tkl.ReLU()
        self.fc2 = tkl.Dense(num_classes)
        self.ac2 = tkl.Softmax()

    def call(self, inputs, *args, **kwargs):
        output = self.preprocess(inputs)
        output = self.block(output)
        output = self.avg_pool(output)
        output = self.dropout(output)
        output = self.flatten(output)
        output = self.fc1(output)
        output = self.ac1(output)
        output = self.fc2(output)
        output = self.ac2(output)
        return output

In [70]:
model = Inception_V1(2)
model.build(input_shape=(None, 64, 64, 3))
model.summary()

Model: "inception__v1_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
preprocess_15 (Preprocess)   multiple                  113600    
_________________________________________________________________
sequential_15 (Sequential)   (None, 16, 16, 160)       129824    
_________________________________________________________________
average_pooling2d_15 (Averag multiple                  0         
_________________________________________________________________
dropout_12 (Dropout)         multiple                  0         
_________________________________________________________________
flatten_12 (Flatten)         multiple                  0         
_________________________________________________________________
dense_24 (Dense)             multiple                  3318016   
_________________________________________________________________
re_lu_83 (ReLU)              multiple             

In [71]:
train_dir = "data/training_set"
test_dir = "data/test_set"
data_gen = ImageDataGenerator(rescale=1.0 / 255)
train_data = data_gen.flow_from_directory(train_dir,
                                          target_size=(64, 64))
test_data = data_gen.flow_from_directory(test_dir,
                                         target_size=(64, 64))
print("数据加载完成")
print(train_data.class_indices)

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.
数据加载完成
{'cats': 0, 'dogs': 1}


In [72]:
epochs = 10
sgd = SGD(learning_rate=0.01,
          momentum=0.05,
          nesterov=True,
          )
model.compile(loss="binary_crossentropy",
              optimizer=sgd,
              metrics=["acc"])
print("模型编译成功")
model.summary()

模型编译成功
Model: "inception__v1_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
preprocess_15 (Preprocess)   multiple                  113600    
_________________________________________________________________
sequential_15 (Sequential)   (None, 16, 16, 160)       129824    
_________________________________________________________________
average_pooling2d_15 (Averag multiple                  0         
_________________________________________________________________
dropout_12 (Dropout)         multiple                  0         
_________________________________________________________________
flatten_12 (Flatten)         multiple                  0         
_________________________________________________________________
dense_24 (Dense)             multiple                  3318016   
_________________________________________________________________
re_lu_83 (ReLU)              multiple      

In [73]:
history = model.fit(train_data,
                    epochs=epochs,
                    validation_data=test_data,
                    batch_size=64)

Epoch 1/10
251/251 [==============================] - 56s 221ms/step - loss: 0.6517 - acc: 0.6437 - val_loss: 1.0170 - val_acc: 0.4998